In [2]:
import pandas as pd
# 5-core-train
df = pd.read_csv('5core/All_Beauty.train.csv')
print("shape of data = ", df.shape)
df[:5]

shape of data =  (2237, 5)


,user_id,parent_asin,rating,timestamp,history
0,AFSKPY37N3C43SOI5IEXEK5JSIYA,B07J3GH1W1,5.0,1547589356557,NaN
1,AFSKPY37N3C43SOI5IEXEK5JSIYA,B07W397QG4,5.0,1593352422858,B07J3GH1W1
2,AFSKPY37N3C43SOI5IEXEK5JSIYA,B07KG1TWP5,5.0,1596473351088,B07J3GH1W1 B07W397QG4
3,AFSKPY37N3C43SOI5IEXEK5JSIYA,B08JTNQFZY,5.0,1617904219785,B07J3GH1W1 B07W397QG4 B07KG1TWP5
4,AFSKPY37N3C43SOI5IEXEK5JSIYA,B07SLFWZKN,3.0,1619737501209,B07J3GH1W1 B07W397QG4 B07KG1TWP5 B08JTNQFZY


In [8]:
from surprise import KNNWithMeans, Reader, Dataset
from surprise.model_selection import train_test_split, GridSearchCV
from surprise.accuracy import rmse

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'parent_asin', 'rating']], reader)
trainset = data.build_full_trainset()

In [9]:
#  Initialize and Train KNN with Means
sim_options = {
    'name': 'cosine',  # Try different similarity measures to see what works best.
    'user_based': True  # Set to False for item-based collaborative filtering.
}

model = KNNWithMeans(sim_options=sim_options)
model.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [10]:
# 5-core-test
test_df = pd.read_csv('5core/All_Beauty.test.csv')
print("shape of data = ", df.shape)
df[:5]

shape of data =  (2237, 5)


,user_id,parent_asin,rating,timestamp,history
0,AFSKPY37N3C43SOI5IEXEK5JSIYA,B07J3GH1W1,5.0,1547589356557,NaN
1,AFSKPY37N3C43SOI5IEXEK5JSIYA,B07W397QG4,5.0,1593352422858,B07J3GH1W1
2,AFSKPY37N3C43SOI5IEXEK5JSIYA,B07KG1TWP5,5.0,1596473351088,B07J3GH1W1 B07W397QG4
3,AFSKPY37N3C43SOI5IEXEK5JSIYA,B08JTNQFZY,5.0,1617904219785,B07J3GH1W1 B07W397QG4 B07KG1TWP5
4,AFSKPY37N3C43SOI5IEXEK5JSIYA,B07SLFWZKN,3.0,1619737501209,B07J3GH1W1 B07W397QG4 B07KG1TWP5 B08JTNQFZY


In [11]:
# Convert DataFrame to list of tuples for testing
testset = list(zip(test_df['user_id'].values, test_df['parent_asin'].values, test_df['rating'].values))

In [12]:
# Test the Model
predictions = model.test(testset)
print("Test RMSE: ", rmse(predictions))


RMSE: 1.4255
Test RMSE:  1.4254651422562015


# Observation
On a 1 to 5 scale, an error of approximately 1.42 means that your predictions are generally off by slightly more than one rating point.

In [14]:
# Use the Model for Recommendations
user_id = 'AFSKPY37N3C43SOI5IEXEK5JSIYA'
item_id = 'B07SLFWZKN'
predicted_rating = model.predict(user_id, item_id)
print(f"Estimated rating for {item_id} by {user_id} is {predicted_rating.est}")

Estimated rating for B07SLFWZKN by AFSKPY37N3C43SOI5IEXEK5JSIYA is 4.037774349833174


# Observation
actual rating is 3.0. error is not trival

In [15]:
# Initialize the SVD algorithm
from surprise import SVD
from surprise import accuracy

model = SVD()
model.fit(trainset)

# Test the model
predictions = model.test(testset)

# Calculate and print RMSE
rmse = accuracy.rmse(predictions)

RMSE: 1.1795


In [16]:
# Use the Model for Recommendations
user_id = 'AFSKPY37N3C43SOI5IEXEK5JSIYA'
item_id = 'B07SLFWZKN'
predicted_rating = model.predict(user_id, item_id)
print(f"Estimated rating for {item_id} by {user_id} is {predicted_rating.est}")

Estimated rating for B07SLFWZKN by AFSKPY37N3C43SOI5IEXEK5JSIYA is 4.070465018238326


In [17]:
# use grid search to improve SVD model
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005], 'reg_all': [0.02, 0.1]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

0.904370009296442
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.02}
